In [1]:
import pandas as pd

TumData=pd.read_csv('Sources/TumDataBattINFO.csv')

In [ ]:
import json
import pandas as pd
import numpy as np


def extract_valid_citations(json_file_path,cell_name):
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    valid_citations= []

    for entry in data:
        if entry.get('state') == 'valid':
            cell_name_entry = entry['filename'].replace('.json', '')
            if cell_name_entry in cell_name:
                valid_citations.append(entry['_id'])
        
    return valid_citations

def fixPG_DOI_ID(citations):
     PG_DOI=pd.read_csv('Sources/pg_doi.csv')
     citations_only_DOI=[]
     for citation in citations:
        if citation.isdigit():
            #print('is digit')
            #print('new DOI ' + PG_DOI.loc[PG_DOI['pgid'] == int(citation), 'doi'].values)
            if not PG_DOI.loc[PG_DOI['pgid'] == int(citation), 'doi'].values[0] == 'nan':
                citations_only_DOI.append(PG_DOI.loc[PG_DOI['pgid'] == int(citation), 'doi'].values[0])
            else:
                citations_only_DOI.append(citation)   
        else:
            citations_only_DOI.append(citation)
     return citations_only_DOI

In [69]:
import json

def create_json_files(df):
    df.reset_index(drop=True, inplace=True)

    for index, row in df.iterrows(): #.iloc[170:171]
        cell_format = row['Format']

        data = {
            "@context": "https://w3id.org/emmo/domain/battery/context",
            "schema:name": row['Part #'],
            "schema:manufacturer": {
                "schema:name": row['Company Name']
            },
            "hasPositiveElectrode": {
                "@type": "PositiveElectrode",
                "hasActiveMaterial": {
                    "@type": row['Chemistry'],
                    "schema:citation": {
                        "@id": "https://github.com/TUMFTM/TruckBatteryDesign/raw/main/Step1_cell_selection/inputs/CellDatabase_v6.xlsx"
                    }
                }
            }}
        
        valid_citations = extract_valid_citations('Sources/valid_hits.json',row['Part #'])
        valid_citations = fixPG_DOI_ID(valid_citations)
        data["schema:citation"]=[{"@id": valid_citations}]
        
        if cell_format.lower() == "cylindricalcase":
            cell_name = row['Part #']
            if "18650" in cell_name:
                data["hasCase"] = [{"@type": "R18650"}]
            elif "21700" in cell_name:
                data["hasCase"] = [{"@type": "21700"}]
            elif "26650" in cell_name:
                data["hasCase"] = [{"@type": "26650"}]    
            
        elif cell_format.lower() in ["pouchcase", "prismaticcase"]:
            data["hasCase"] = [{"@type": row['Cell Shape']}]

        data["hasProperty"]=[
                {
                    "@type": ["NominalCapacity", "ConventionalProperty"],
                    "hasNumericalPart": {
                        "@type": "Real",
                        "hasNumericalValue": row['Max Capacity (AH)']
                    },
                    "hasMeasurementUnit": "emmo:AmpereHour",
                    "schema:citation": {"@id": "https://github.com/TUMFTM/TruckBatteryDesign/raw/main/Step1_cell_selection/inputs/CellDatabase_v6.xlsx"}
                },
                {
                    "@type": ["CycleLife", "ConventionalProperty"],
                    "hasNumericalPart": {
                        "@type": "Real",
                        "hasNumericalValue": row['Cycle Life']
                    },
                    "schema:citation": {"@id": "https://github.com/TUMFTM/TruckBatteryDesign/raw/main/Step1_cell_selection/inputs/CellDatabase_v6.xlsx"}
                },
                {
                    "@type": ["NominalVoltage", "ConventionalProperty"],
                    "hasNumericalPart": {
                        "@type": "Real",
                        "hasNumericalValue": row['Nominal Voltage (V)']
                    },
                    "hasMeasurementUnit": "emmo:Volt",
                    "schema:citation": {"@id": "https://github.com/TUMFTM/TruckBatteryDesign/raw/main/Step1_cell_selection/inputs/CellDatabase_v6.xlsx"}
                },
                {
                    "@type": ["UpperVoltageLimit", "ConventionalProperty"],
                    "hasNumericalPart": {
                        "@type": "Real",
                        "hasNumericalValue": row['Open Circuit Voltage (V)']
                    },
                    "hasMeasurementUnit": "emmo:Volt",
                    "schema:citation": {"@id": "https://github.com/TUMFTM/TruckBatteryDesign/raw/main/Step1_cell_selection/inputs/CellDatabase_v6.xlsx"}
                },
                {
                    "@type": ["LowerVoltageLimit", "ConventionalProperty"],
                    "hasNumericalPart": {
                        "@type": "Real",
                        "hasNumericalValue": row['Cut Off Voltage (V)']
                    },
                    "hasMeasurementUnit": "emmo:Volt",
                    "schema:citation": {"@id": "https://github.com/TUMFTM/TruckBatteryDesign/raw/main/Step1_cell_selection/inputs/CellDatabase_v6.xlsx"}
                },
                {
                    "@type": ["DischargeCurrent", "ConventionalProperty"],
                    "hasNumericalPart": {
                        "@type": "Real",
                        "hasNumericalValue": row['Standard Discharge Current (A)']
                    },
                    "hasMeasurementUnit": "emmo:Ampere",
                    "schema:citation": {"@id": "https://github.com/TUMFTM/TruckBatteryDesign/raw/main/Step1_cell_selection/inputs/CellDatabase_v6.xlsx"}
                },
                {
                    "@type": ["MaximumContinuousDischargeCurrent", "ConventionalProperty"],
                    "hasNumericalPart": {
                        "@type": "Real",
                        "hasNumericalValue": row['Max Constant Discharge Current (A)']
                    },
                    "hasMeasurementUnit": "emmo:Ampere",
                    "schema:citation": {"@id": "https://github.com/TUMFTM/TruckBatteryDesign/raw/main/Step1_cell_selection/inputs/CellDatabase_v6.xlsx"}
                },
                {
                    "@type": ["Mass", "ConventionalProperty"],
                    "hasNumericalPart": {
                        "@type": "Real",
                        "hasNumericalValue": row['Weight (gr)'] / 1000
                    },
                    "hasMeasurementUnit": "emmo:Kilogram",
                    "schema:citation": {"@id": "https://github.com/TUMFTM/TruckBatteryDesign/raw/main/Step1_cell_selection/inputs/CellDatabase_v6.xlsx"}
                },
                {
                    "@type": ["ChargingCurrent", "ConventionalProperty"],
                    "hasNumericalPart": {
                        "@type": "Real",
                        "hasNumericalValue": row['Standard Charge Current (A)']
                    },
                    "hasMeasurementUnit": "emmo:Ampere",
                    "schema:citation": {"@id": "https://github.com/TUMFTM/TruckBatteryDesign/raw/main/Step1_cell_selection/inputs/CellDatabase_v6.xlsx"}
                }
            ]
        
        if cell_format.lower() == "cylindricalcase":
            data["hasProperty"].extend([
                {
                    "@type": ["Height", "ConventionalProperty"],
                    "hasNumericalPart": {
                        "@type": "Real",
                        "hasNumericalValue": row['Height (mm)'] / 1000
                    },
                    "hasMeasurementUnit": "emmo:Metre",
                    "schema:citation": {"@id": "https://github.com/TUMFTM/TruckBatteryDesign/raw/main/Step1_cell_selection/inputs/CellDatabase_v6.xlsx"}
                },
                {
                    "@type": ["Diameter", "ConventionalProperty"],
                    "hasNumericalPart": {
                        "@type": "Real",
                        "hasNumericalValue": row['Diameter (mm)'] / 1000
                    },
                    "hasMeasurementUnit": "emmo:Metre",
                    "schema:citation": {"@id": "https://github.com/TUMFTM/TruckBatteryDesign/raw/main/Step1_cell_selection/inputs/CellDatabase_v6.xlsx"}
                }
            ])            
        elif cell_format.lower() in ["pouchcase", "prismaticcase"]:
            data["hasProperty"].extend([
                {
                    "@type": ["Height", "ConventionalProperty"],
                    "hasNumericalPart": {
                        "@type": "Real",
                        "hasNumericalValue": row['Height (mm)'] / 1000
                    },
                    "hasMeasurementUnit": "emmo:Metre",
                    "schema:citation": {"@id": "https://github.com/TUMFTM/TruckBatteryDesign/raw/main/Step1_cell_selection/inputs/CellDatabase_v6.xlsx"}
                },
                {
                    "@type": ["Width", "ConventionalProperty"],
                    "hasNumericalPart": {
                        "@type": "Real",
                        "hasNumericalValue": row['Width (mm)'] / 1000
                    },
                    "hasMeasurementUnit": "emmo:Metre",
                    "schema:citation": {"@id": "https://github.com/TUMFTM/TruckBatteryDesign/raw/main/Step1_cell_selection/inputs/CellDatabase_v6.xlsx"}
                },
                {
                    "@type": ["Length", "ConventionalProperty"],
                    "hasNumericalPart": {
                        "@type": "Real",
                        "hasNumericalValue": row['Length (mm)'] / 1000
                    },
                    "hasMeasurementUnit": "emmo:Metre",
                    "schema:citation": {"@id": "https://github.com/TUMFTM/TruckBatteryDesign/raw/main/Step1_cell_selection/inputs/CellDatabase_v6.xlsx"}
                }
            ])



        fileName = 'BatteryTypeJson/' + row['Company Name'] + '_' + row['Part #'].replace('/', '_') + '.json'
        with open(fileName.replace(' ', '_'), 'w') as json_file:
            json.dump(data, json_file, indent=4)
        #return data

# Create the JSON files
create_json_files(TumData)
#single_data = create_json_files(TumData)
#print(json.dumps(single_data, indent=4))


FileNotFoundError: [Errno 2] No such file or directory: 'BatteryTypeJson/BMZ_/_Sony_BMZ_21700_50EL.json'

In [65]:



# Display the results

#print(valid_citations)


['10.1007_s42452-020-2675-6', '10.1016_j.jpowsour.2021.229463', '10.1016_j.sna.2021.113061', '10.3390_en11051073', '10.1016_j.egyr.2023.04.148', '10.1155_2017_2579084', '10.1016_j.compeleceng.2021.107306', '10.1016_j.applthermaleng.2017.04.017', '10.1016_j.applthermaleng.2020.116338', '10.1016_j.renene.2019.08.077', '10.1016_j.energy.2023.128126', '10.1016_j.jpowsour.2020.228189', '10.1016_j.egyr.2021.11.089', '10.3390_en10091284', '10.1016_j.est.2021.102466', '10.1016_j.est.2022.105977', '10.1016_j.apenergy.2022.120037', nan, '10.22149_teee.v4i1.136', '10.1016_j.est.2021.102457', '10.1016_j.est.2020.101552', '10.1016_j.jpowsour.2019.227466', '10.1016_j.est.2020.101789', '10.1016_j.est.2022.105796', '10.1016_j.jpowsour.2020.228690', '10.1016_j.micpro.2020.103711', '10.1016_j.est.2022.105345', '10.1016_j.est.2023.108047', '10.1016_j.energy.2021.119901', '10.1016_j.est.2016.04.007', '10.1016_j.engfailanal.2019.104290', '10.1016_j.ijepes.2021.107760', '10.1016_j.jpowsour.2015.12.083', '10

In [49]:
valid_citations[1].isdigit()

False

In [60]:
PG_DOI=pd.read_csv('Sources/pg_doi.csv')
#PG_DOI['doi'].loc[PG_DOI['pgid'] == 66761629]
PG_DOI.loc[PG_DOI['pgid'] == 66761629, 'doi'].values[0]
#PG_DOI['doi'].loc[75944]

'10.1109_tcst.2020.2976036'